In [1]:
import sys

In [2]:
from pytket import Circuit
from tket.circuit import Tk2Circuit
# from tket.protocol import CircuitReplacer

In [3]:
sys.path.append("../../../tket2-rewriting/tket2_rewriting")
from circuit_generation.gadget import CXGadgetBenchmarkCircuit,SimpleLabelFriendlyCircuit
from pytket.circuit.display import render_circuit_jupyter
import numpy as np
from pytket.passes import DecomposeBoxes, AutoRebase
from pytket import OpType

# Original circuit to be compiled
orig_circuit = SimpleLabelFriendlyCircuit(width=5, depth=10, rng=np.random.default_rng(0))
orig_circuit.n_2qb_gates()

20

In [4]:
orig_circuit

[CX q[0], q[4]; Rz(1.42351) q[1]; Rz(2.65582) q[2]; Rz(0.257444) q[0]; Rz(1.23166) q[1]; Rx(0.780895) q[2]; Rx(1.10993) q[0]; Rz(0.570218) q[1]; CX q[0], q[4]; Rz(3.89692) q[1]; CX q[0], q[3]; Rz(0.614896) q[1]; Rz(0.533245) q[4]; Rz(1.38723) q[0]; Rx(2.11121) q[1]; Rx(0.58456) q[0]; CX q[0], q[3]; CX q[0], q[2]; Rz(2.26565) q[0]; CX q[0], q[2]; CX q[0], q[3]; Rz(1.50711) q[2]; Rz(3.05259) q[0]; Rx(2.11189) q[2]; CX q[0], q[3]; CX q[0], q[4]; Rz(2.82957) q[3]; Rx(3.9156) q[0]; Rz(0.222268) q[3]; CX q[0], q[4]; Rx(1.91947) q[3]; CX q[0], q[2]; Rz(1.46004) q[3]; Rz(0.662846) q[4]; Rz(0.326859) q[0]; Rx(0.518991) q[3]; CX q[0], q[2]; CX q[0], q[4]; Rx(0.580537) q[2]; Rz(1.99787) q[0]; Rz(1.84975) q[2]; CX q[0], q[4]; Rx(0.599331) q[2]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[1]; Rz(0.762347) q[4]; Rx(3.38622) q[0]; CX q[0], q[1]; ]

In [5]:
sys.path.append("../../../tket2-rewriting/tket2_rewriting")

from cx_gadget.cx_gadget import CXGadget
from zzphase_flip.zzphase_flip import ZZPhaseFlip
from three_qubit_squash.three_qubit_squash_gadget import Squash3Gadget
from KAK.kak_gadget import KAKGadget

In [6]:
circuit_tk2 = Tk2Circuit(orig_circuit)
circuit_tk2

## Existing gadgets

In [7]:
from tket.optimiser import BadgerOptimiser
from time import time
from pytket.passes import GreedyPauliSimp, FullPeepholeOptimise

circuit = orig_circuit.copy()

opt = BadgerOptimiser(
    [
        # ZZPhaseFlip(),
        # CXGadget(),
        Squash3Gadget(),
        # KAKGadget()
    ]
)

In [ ]:
# Optimise circuit and time the optimisation - this is doing something
start_time = time()
opt_circ = opt.optimise(circuit_tk2)
print(f"time taken: {time() - start_time}")
# render_circuit_jupyter(opt_circ)

1 qubit gate, if CX : 0->1, 1->0, if I: 0->0, 1->1, otherwise ?
2 qubit gate, if CX: control 0 -> target stays as is, control 1 -> target swaps 0->1 or 1->0, control label is unchanged
else: control 0 -> target stays as is, control 1 -> target ?, control label is unchanged

In [ ]:
opt_circ1 = opt_circ.to_tket1()
type(opt_circ1) , type(circuit)

(pytket._tket.circuit.Circuit, pytket._tket.circuit.Circuit)

In [ ]:
print(f"Two qubit gates: {opt_circ1.n_2qb_gates()} vs previous: {circuit.n_2qb_gates()}")

Two qubit gates: 16 vs previous: 20


In [ ]:
orig_circuit

[CX q[0], q[4]; Rz(1.42351) q[1]; Rz(2.65582) q[2]; Rz(0.257444) q[0]; Rz(1.23166) q[1]; Rx(0.780895) q[2]; Rx(1.10993) q[0]; Rz(0.570218) q[1]; CX q[0], q[4]; Rz(3.89692) q[1]; CX q[0], q[3]; Rz(0.614896) q[1]; Rz(0.533245) q[4]; Rz(1.38723) q[0]; Rx(2.11121) q[1]; Rx(0.58456) q[0]; CX q[0], q[3]; CX q[0], q[2]; Rz(2.26565) q[0]; CX q[0], q[2]; CX q[0], q[3]; Rz(1.50711) q[2]; Rz(3.05259) q[0]; Rx(2.11189) q[2]; CX q[0], q[3]; CX q[0], q[4]; Rz(2.82957) q[3]; Rx(3.9156) q[0]; Rz(0.222268) q[3]; CX q[0], q[4]; Rx(1.91947) q[3]; CX q[0], q[2]; Rz(1.46004) q[3]; Rz(0.662846) q[4]; Rz(0.326859) q[0]; Rx(0.518991) q[3]; CX q[0], q[2]; CX q[0], q[4]; Rx(0.580537) q[2]; Rz(1.99787) q[0]; Rz(1.84975) q[2]; CX q[0], q[4]; Rx(0.599331) q[2]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[4]; CX q[0], q[1]; Rz(0.762347) q[4]; Rx(3.38622) q[0]; CX q[0], q[1]; ]

In [ ]:
opt_circ1

[CX q[0], q[4]; Rz(1.42351) q[1]; Rz(2.65582) q[2]; Rz(0.257444) q[0]; Rz(1.23166) q[1]; Rx(0.780895) q[2]; Rx(1.10993) q[0]; Rz(0.570218) q[1]; CX q[0], q[4]; Rz(3.89692) q[1]; CX q[0], q[3]; Rz(0.614896) q[1]; Rz(0.533245) q[4]; Rz(1.38723) q[0]; Rx(2.11121) q[1]; Rx(0.58456) q[0]; CX q[0], q[3]; CX q[0], q[2]; Rz(2.26565) q[0]; CX q[0], q[2]; CX q[0], q[3]; Rz(1.50711) q[2]; Rz(3.05259) q[0]; Rx(2.11189) q[2]; CX q[0], q[3]; CX q[0], q[4]; Rz(2.82957) q[3]; Rx(3.9156) q[0]; Rz(0.222268) q[3]; CX q[0], q[4]; Rx(1.91947) q[3]; CX q[0], q[2]; Rz(1.46004) q[3]; Rz(0.662846) q[4]; Rz(0.326859) q[0]; Rx(0.518991) q[3]; CX q[0], q[2]; CX q[0], q[4]; Rx(0.580537) q[2]; Rz(1.99787) q[0]; Rz(1.84975) q[2]; CX q[0], q[4]; Rx(0.599331) q[2]; CX q[0], q[1]; Rz(0.762347) q[4]; Rx(3.38622) q[0]; CX q[0], q[1]; ]

In [ ]:
# Follow with FullPeepholeOptimise
FullPeepholeOptimise().apply(opt_circ1)
print(f"Two qubit gates: {opt_circ1.n_2qb_gates()}")

Two qubit gates: 8


In [ ]:
opt_circ1

[TK1(0, 0, 0.257444) q[0]; TK1(0, 0.111207, 1.7372) q[1]; TK1(0.877019, 3.74199, 0.149414) q[2]; CX q[0], q[4]; TK1(1.38723, 1.10993, 0) q[0]; CX q[0], q[4]; CX q[0], q[3]; TK1(0, 0, 0.533245) q[4]; TK1(1.31825, 0.58456, 0) q[0]; CX q[0], q[3]; CX q[0], q[4]; TK1(3.08021, 3.47163, 1.51787) q[3]; TK1(0.324728, 1.9156, 0) q[0]; CX q[0], q[4]; CX q[0], q[1]; TK1(0, 0, 1.42519) q[4]; TK1(0, 1.38622, 0) q[0]; CX q[0], q[1]; ]

In [ ]:
# With GreedyPauliSimp the result is not as good
circuit = orig_circuit.copy()
GreedyPauliSimp().apply(circuit)
print(f"Two qubit gates: {circuit.n_2qb_gates()}")
# render_circuit_jupyter(circuit)

Two qubit gates: 8


In [ ]:
opt_circ1

[TK1(0, 0, 0.257444) q[0]; TK1(0, 0.111207, 1.7372) q[1]; TK1(0.877019, 3.74199, 0.149414) q[2]; CX q[0], q[4]; TK1(1.38723, 1.10993, 0) q[0]; CX q[0], q[4]; CX q[0], q[3]; TK1(0, 0, 0.533245) q[4]; TK1(1.31825, 0.58456, 0) q[0]; CX q[0], q[3]; CX q[0], q[4]; TK1(3.08021, 3.47163, 1.51787) q[3]; TK1(0.324728, 1.9156, 0) q[0]; CX q[0], q[4]; CX q[0], q[1]; TK1(0, 0, 1.42519) q[4]; TK1(0, 1.38622, 0) q[0]; CX q[0], q[1]; ]

In [ ]:
# With FullPeephole the result is not as good
circuit = orig_circuit.copy()
FullPeepholeOptimise().apply(circuit)
print(f"Two qubit gates: {circuit.n_2qb_gates()}")
# render_circuit_jupyter(circuit)

Two qubit gates: 8


In [ ]:
# With both the result is not as good
circuit = orig_circuit.copy()
GreedyPauliSimp().apply(circuit)
FullPeepholeOptimise().apply(circuit)
print(f"Two qubit gates: {circuit.n_2qb_gates()}")
# render_circuit_jupyter(circuit)

Two qubit gates: 8


In [ ]:
from pytket import Circuit, OpType
from pytket.passes import DecomposeBoxes, AutoRebase
from pytket import OpType
from tket.circuit import Tk2Circuit
from tket.optimiser import BadgerOptimiser
# from three_qubit_squash.three_qubit_squash_gadget import Squash3Gadget

def boundary_ports_between_barriers(circ: Circuit):
    """
    Return (ports, (start_idx, end_idx), circ_used).
    If circ has <2 barriers, it builds a bracketed copy: [BARRIER] + circ + [BARRIER].
    """
    def _compute_ports(c, sidx, eidx):
        cmds = list(c.get_commands())
        mid = cmds[sidx+1:eidx]
        touched = set()
        for cmd in mid:
            for u in cmd.args:
                touched.add(u)
        return [q for q in c.qubits if q in touched]

    cmds = list(circ.get_commands())
    barrier_idxs = [i for i, cmd in enumerate(cmds) if cmd.op.type == OpType.Barrier]

    if len(barrier_idxs) >= 2:
        start_idx, end_idx = barrier_idxs[0], barrier_idxs[-1]
        return _compute_ports(circ, start_idx, end_idx), (start_idx, end_idx), circ

    # Auto-bracket: [BARRIER] + circ + [BARRIER]
    c2 = Circuit(circ.n_qubits)
    c2.add_barrier(c2.qubits)
    # qubit mapping
    c2.add_circuit(circ, c2.qubits)
    c2.add_barrier(c2.qubits)

    cmds2 = list(c2.get_commands())
    barrier_idxs2 = [i for i, cmd in enumerate(cmds2) if cmd.op.type == OpType.Barrier]
    start_idx, end_idx = barrier_idxs2[0], barrier_idxs2[-1]
    return _compute_ports(c2, start_idx, end_idx), (start_idx, end_idx), c2

# --- usage ---
orig = SimpleLabelFriendlyCircuit(6, 6)

cprep = orig.copy()
DecomposeBoxes().apply(cprep)
AutoRebase({OpType.CX, OpType.Rz, OpType.Rx}).apply(cprep)

before_ports, (sidx, eidx), circ_used = boundary_ports_between_barriers(cprep)

opt = BadgerOptimiser([Squash3Gadget()])
tk2 = Tk2Circuit(circ_used.copy())
_ = opt.optimise(tk2)
after = tk2.to_tket1()

after_ports, _, _ = boundary_ports_between_barriers(after)

print("Boundary ports before:", before_ports)
print("Boundary ports after :", after_ports)
print("Ports preserved?      ", before_ports == after_ports)



Boundary ports before: [q[0], q[1], q[2], q[3], q[4], q[5]]
Boundary ports after : [q[0], q[1], q[2], q[3], q[4], q[5]]
Ports preserved?       True
